### Clean Name Entity Merge Result

In [ ]:
# !pip install modin[ray]

In [ ]:
#import ray  # ayrı olarak çalıştırılmalı
#ray.init() 
# Not: Eğer multiprocessing kullanılacaksa ray init kullanılmamalı

In [ ]:
#import modin.pandas as pd
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_rows', 50)

#### Clean One Character And Digit

In [ ]:
df = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Pyspark/Result/Sentence/Spark Turkish Sentence Merge/Clean_Spark_Sentence_Merge_Result.csv")
df = df.iloc[:,0:2]
df

In [ ]:
data_kind = "sentence" # word, twogram ...
file_name = "Sentence_More_Than_Two_Word"

In [ ]:
df = df[~df.iloc[:,0].str.contains(fr"fff", na=True)]
df.reset_index(drop=True, inplace=True)
df

In [ ]:
def clean_num(var):
    var_list = re.findall("\w+", var)
    try:
        if var_list[0][0].isdigit() or (len(var_list) <= 1 and len(var_list[0]) <= 1):
            return "digit or one character"
        else:
            return var
    except:
        return "not digit or character"

In [ ]:
df.iloc[:,0] = df.iloc[:,0].apply(lambda x : clean_num(x))
df

In [ ]:
df[df.iloc[:,0] == "digit or one character"]

In [ ]:
df[df.iloc[:,0] == "not digit or character"]

In [ ]:
df.drop(df[df.iloc[:,0] == "digit or one character"].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df

In [ ]:
df.drop(df[df.iloc[:,0] == "not digit or character"].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df

In [ ]:
df.to_csv(f"{file_name}.csv", index=False)

#### Include Name Entity

In [ ]:
df_no_digit = pd.read_csv(f"{file_name}.csv")
df_no_digit = df_no_digit.iloc[:,0:2].head(10000)
#df_no_digit = df_no_digit.iloc[:,0:2]
df_no_digit

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re

In [ ]:
# help(multiprocessing)

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [ ]:
name_entity_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Pyspark/Result/Name Entity/Special_Name_Unique.xlsx"

In [ ]:
df_ne = pd.read_excel(f"{name_entity_path}")
df_ne = df_ne.head(100)
df_ne

In [ ]:
ne_list = [x.lower() for x in df_ne.iloc[:,0]]

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
special_name_list = manager.list()

def find_special_name(name_ent):
    name_in_df = df_no_digit[df_no_digit.iloc[:,0].str.contains(fr"(?:\s|^){name_ent}(?:\s|$)", na=True)]
    ne_list = name_in_df.iloc[:,0].to_list()
    for j in ne_list:
        special_name_list.append(j) 

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(find_special_name, ne_list) # string_word liste

In [ ]:
list_special_name = list(special_name_list)
df_special_name = pd.DataFrame(list_special_name)
df_special_name.rename(columns={0:f"{data_kind}"}, inplace=True)
df_special_name

In [ ]:
df_special_freq = pd.merge(df_no_digit, df_special_name, how="inner", on=f"{data_kind}")
df_special_freq

In [ ]:
df_special_freq.to_csv(f"{file_name}_Include_Name_Entity_Result.csv", index=False)

#### Extract Name Entity From Data

In [ ]:
df_no_digit = pd.read_csv(f"{file_name}.csv")
df_no_digit = df_no_digit.iloc[:,0:2].head(10000)
#df_no_digit = df_no_digit.iloc[:,0:2]
df_no_digit

In [ ]:
clean_set = set(df_no_digit.iloc[:,0])

In [ ]:
df_special_freq = pd.read_csv(f"{file_name}_Include_Name_Entity_Result.csv")
df_special_freq

In [ ]:
name_set = set(df_special_name.iloc[:,0])

In [ ]:
df_new = pd.DataFrame(clean_set.difference(name_set))
df_new.rename(columns={0:f"{data_kind}"}, inplace=True)
df_new.reset_index(drop=True, inplace=True)
df_new

In [ ]:
df_new_freq = pd.merge(df_no_digit, df_new, how="inner", on=f"{data_kind}")
df_new_freq

In [ ]:
df_new_freq.to_csv(f"{file_name}_Without_Name_Entity.csv", index=False)

#### Name Entity Change Same Name

In [ ]:
import ray  # ayrı olarak çalıştırılmalı
ray.init() 

In [ ]:
import modin.pandas as pd

In [ ]:
df_special_freq = pd.read_csv(f"{file_name}_Include_Name_Entity_Result.csv")
df_special_freq

In [ ]:
df_ne = pd.read_excel(f"{name_entity_path}")
#df_ne = df_ne.head(100)
df_ne

In [ ]:
ne_list = [x.lower() for x in df_ne.iloc[:,0]]

In [ ]:
def replace_name(x):
    for i in ne_list:
        #x = re.sub(fr"(?:\s|^){i}", " ali ", x)
        x = re.sub(fr"(?:\s|^){i}(?:\s|$)", " fikret ", x)
        x = x.strip()        
    return x

In [ ]:
df_special_freq.iloc[:,0] = df_special_freq.iloc[:,0].apply(lambda x : replace_name(x))
df_special_freq.drop_duplicates(inplace=True)
df_special_freq

In [ ]:
df_spec_sum = df_special_freq.groupby(df_special_freq.iloc[:,0])[["frequency"]].sum()
df_spec_sum = df_spec_sum.sort_values(by="frequency", ascending=False)
df_spec_sum.reset_index(inplace=True)
df_spec_sum

In [ ]:
df_spec_sum.to_csv(f"{file_name}_Name_Entity_Change_Same.csv", index=False)

#### Concat Without Name Entity And Change Same

In [ ]:
df_new_freq= pd.read_csv(f"{file_name}_Without_Name_Entity.csv")
df_new_freq

In [ ]:
df_spec_sum = pd.read_csv(f"{file_name}_Name_Entity_Change_Same.csv")
df_spec_sum

In [ ]:
df_clean_merge_result = pd.concat([df_new_freq, df_spec_sum], axis=0)
df_clean_merge_result = df_clean_merge_result.sort_values(by="frequency", ascending=False)
df_clean_merge_result

In [ ]:
df_clean_merge_result.to_csv(f"{file_name}2.csv", index=False)